<a href="https://colab.research.google.com/github/linhlinhle997/ner-recognition/blob/features%2F02-ner-recog/ner_recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical NER using BERT

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatib

## Data preparation

In [6]:
#https://figshare.com/articles/dataset/MACCROBAT2018/9764942
!unzip /content/MACCROBAT2018.zip -d /content/MACCROBAT2018

Archive:  /content/MACCROBAT2018.zip
  inflating: /content/MACCROBAT2018/15939911.ann  
  inflating: /content/MACCROBAT2018/15939911.txt  
  inflating: /content/MACCROBAT2018/16778410.ann  
  inflating: /content/MACCROBAT2018/16778410.txt  
  inflating: /content/MACCROBAT2018/17803823.ann  
  inflating: /content/MACCROBAT2018/17803823.txt  
  inflating: /content/MACCROBAT2018/18236639.ann  
  inflating: /content/MACCROBAT2018/18236639.txt  
  inflating: /content/MACCROBAT2018/18258107.ann  
  inflating: /content/MACCROBAT2018/18258107.txt  
  inflating: /content/MACCROBAT2018/18416479.ann  
  inflating: /content/MACCROBAT2018/18416479.txt  
  inflating: /content/MACCROBAT2018/18561524.ann  
  inflating: /content/MACCROBAT2018/18561524.txt  
  inflating: /content/MACCROBAT2018/18666334.ann  
  inflating: /content/MACCROBAT2018/18666334.txt  
  inflating: /content/MACCROBAT2018/18787726.ann  
  inflating: /content/MACCROBAT2018/18787726.txt  
  inflating: /content/MACCROBAT2018/18815636.

In [7]:
import os
from typing import List, Dict, Tuple

class Preprocessing_Maccrobat:
    def __init__(self, dataset_folder, tokenizer):
        self.file_ids = [f.split(".")[0] for f in os.listdir(dataset_folder) if f.endswith('.txt')]

        self.text_files = [f+".txt" for f in self.file_ids]
        self.anno_files = [f+".ann" for f in self.file_ids]

        self.num_samples = len(self.file_ids)

        self.texts: List[str] = []
        for i in range(self.num_samples):
            file_path = os.path.join(dataset_folder, self.text_files[i])
            with open(file_path, 'r') as f:
                self.texts.append(f.read())

        self.tags: List[Dict[str, str]] = []
        for i in range(self.num_samples):
            file_path = os.path.join(dataset_folder, self.anno_files[i])
            with open(file_path, 'r') as f:
                text_bound_ann = [t.split("\t") for t in f.read().split("\n") if t.startswith("T")]
                text_bound_lst = []
                for text_b in text_bound_ann:
                    label = text_b[1].split(" ")
                    try:
                        _ = int(label[1])
                        _ = int(label[2])
                        tag = {
                            "text": text_b[-1],
                            "label": label[0],
                            "start": label[1],
                            "end": label[2]
                        }
                        text_bound_lst.append(tag)
                    except:
                        pass

                self.tags.append(text_bound_lst)
        self.tokenizer = tokenizer

    def process(self) -> Tuple[List[List[str]], List[List[str]]]:
        input_texts = []
        input_labels = []

        for idx in range(self.num_samples):
            full_text = self.texts[idx]
            tags = self.tags[idx]

            label_offset = []
            continuous_label_offset = []
            for tag in tags:
                offset = list(range(int(tag["start"]), int(tag["end"])+1))
                label_offset.append(offset) # 345
                continuous_label_offset.extend(offset) #  345

            all_offset = list(range(len(full_text)))
            zero_offset = [offset for offset in all_offset if offset not in continuous_label_offset]
            zero_offset = Preprocessing_Maccrobat.find_continuous_ranges(zero_offset) # 012 67

            self.tokens = []
            self.labels = []
            self._merge_offset(full_text, tags, zero_offset, label_offset)
            assert len(self.tokens) == len(self.labels), f"Length of tokens and labels are not equal"

            input_texts.append(self.tokens)
            input_labels.append(self.labels)

        return input_texts, input_labels

    def _merge_offset(self, full_text, tags, zero_offset, label_offset):
        # zero: [[0,1,2], [6,7]] label: [[3,4,5]]
        i = j = 0
        while i < len(zero_offset) and j < len(label_offset):
            if zero_offset[i][0] < label_offset[j][0]:
                self._add_zero(full_text, zero_offset, i)
                i += 1
            else:
                self._add_label(full_text, label_offset, j, tags)
                j += 1

        while i < len(zero_offset):
            self._add_zero(full_text, zero_offset, i)
            i += 1

        while j < len(label_offset):
            self._add_label(full_text, label_offset, j, tags)
            j += 1

    def _add_zero(self, full_text, offset, index):
        start, *_ ,end =  offset[index] if len(offset[index]) > 1 else (offset[index][0], offset[index][0]+1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            ["O"]*len(text_tokens)
        )

    def _add_label(self, full_text, offset, index, tags):
        start, *_ ,end =  offset[index] if len(offset[index]) > 1 else (offset[index][0], offset[index][0]+1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            [f"B-{tags[index]['label']}"] + [f"I-{tags[index]['label']}"]*(len(text_tokens)-1)
        )

    @staticmethod
    def build_label2id(tokens: List[List[str]]):
        label2id = {}
        id_counter = 0
        for token in [token for sublist in tokens for token in sublist]:
            if token not in label2id:
                label2id[token] = id_counter
                id_counter += 1
        return label2id

    @staticmethod
    def find_continuous_ranges(data: List[int]): # [0, 1, 2, 6, 7]
        if not data:
            return []
        ranges = []
        start = data[0] # 0
        prev = data[0] # 0
        for number in data[1:]: # [1, 2, 6, 7]
            if number != prev + 1:
                ranges.append(list(range(start, prev + 1)))
                start = number
            prev = number
        ranges.append(list(range(start, prev + 1)))
        return ranges

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")

dataset_folder = "/content/MACCROBAT2018"

Maccrobat_builder = Preprocessing_Maccrobat(dataset_folder, tokenizer)
input_texts, input_labels = Maccrobat_builder.process()

label2id = Preprocessing_Maccrobat.build_label2id(input_labels)
id2label = {v: k for k, v in label2id.items()}

## Dataloader

In [9]:
from sklearn.model_selection import train_test_split

inputs_train, inputs_val, labels_train, labels_val = train_test_split(
    input_texts, input_labels, test_size=0.2, random_state=42
)

In [10]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 512

class NER_Dataset(Dataset):
    def __init__(self, input_texts, input_labels, tokenizer, label2id, max_len=MAX_LEN):
        super().__init__()
        self.tokens = input_texts
        self.labels = input_labels
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        input_token = self.tokens[idx]
        label_token = [self.label2id[label] for label in self.labels[idx]]

        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        attentio_mask = [1] * len(input_token)

        input_ids = self.pad_and_truncate(input_token, pad_id=self.tokenizer.pad_token_id)
        labels = self.pad_and_truncate(label_token, pad_id=0)
        attention_mask = self.pad_and_truncate(attentio_mask, pad_id=0)

        return {
            "input_ids": torch.as_tensor(input_ids),
            "labels": torch.as_tensor(labels),
            "attention_mask": torch.as_tensor(attention_mask)
        }

    def pad_and_truncate(self, inputs: List[int], pad_id: int):
        if len(inputs) < self.max_len:
            padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
        else:
            padded_inputs = inputs[:self.max_len]
        return padded_inputs

    def label2id(self, labels: List[str]):
        return [self.label2id[label] for label in labels]

train_set = NER_Dataset(inputs_train, labels_train, tokenizer, label2id)
val_set = NER_Dataset(inputs_val, labels_val, tokenizer, label2id)

## Model

In [11]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "d4data/biomedical-ner-all",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at d4data/biomedical-ner-all and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([84]) in the checkpoint and torch.Size([83]) in the model instantiated
- classifier.weight: found shape torch.Size([84, 768]) in the checkpoint and torch.Size([83, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training

In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [13]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mask = labels != 0
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="adamw_torch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.699941,0.313327
2,No log,1.032434,0.604157
3,No log,0.817745,0.675875
4,No log,0.711539,0.726986
5,No log,0.646811,0.746513
6,No log,0.629853,0.757671
7,No log,0.616376,0.762350
8,No log,0.625919,0.770449
9,No log,0.646779,0.767749
10,No log,0.643845,0.772249


TrainOutput(global_step=200, training_loss=0.4273954772949219, metrics={'train_runtime': 269.0007, 'train_samples_per_second': 11.896, 'train_steps_per_second': 0.743, 'total_flos': 418702245888000.0, 'train_loss': 0.4273954772949219, 'epoch': 20.0})

## Inference

In [15]:
test_sentence = """A 48 year-old female presented with vaginal bleeding and abnormal
Pap smears. Upon diagnosis of invasive non-keratinizing SCC of the cervix, she
underwent a radical hysterectomy with salpingo-oophorectomy which demonstrated
positive spread to the pelvic lymph nodes and the parametrium. Pathological
examination revealed that the tumour also extensively involved the lower uterine
segment."""

input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])
input = input.to("cuda")

outputs = model(input)
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

for token, pred in zip(test_sentence.split(), preds):
    print(f"{token}\t{id2label[pred]}")

A	O
48	O
year-old	I-Age
female	B-Sex
presented	O
with	O
vaginal	O
bleeding	B-Sign_symptom
and	O
abnormal	B-Detailed_description
Pap	O
smears.	O
Upon	O
diagnosis	O
of	O
invasive	B-Detailed_description
non-keratinizing	O
SCC	O
of	O
the	O
cervix,	O
she	O
underwent	O
a	O
radical	O
hysterectomy	O
with	O
salpingo-oophorectomy	O
which	O
demonstrated	O
positive	B-Lab_value
spread	O
to	O
the	O
pelvic	O
lymph	O
nodes	O
and	O
the	O
parametrium.	O
Pathological	I-Diagnostic_procedure
examination	O
revealed	O
that	O
the	O
tumour	O
also	O
extensively	O
involved	O
the	O
lower	O
uterine	O
segment.	O


In [16]:
def merge_entity(sentence, preds, model):
    merged_list = []
    prev_value = None
    temp_keys = []

    for key, value in zip(sentence.split(), preds):
        value = model.config.id2label[value].split('-')[-1]
        if value == "O":
            if temp_keys:
                merged_list.append((prev_value, ", ".join(temp_keys)))
                temp_keys = []
            merged_list.append((value, key))
            prev_value = None
        elif value == prev_value:
            temp_keys.append(key)
        else:
            if temp_keys:
                merged_list.append((prev_value, " ".join(temp_keys)))
            temp_keys = [key]
            prev_value = value

    if temp_keys:
        merged_list.append((prev_value, ", ".join(temp_keys)))
    return merged_list


def inference(sentence, model):
    input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])
    input = input.to("cuda")
    with torch.no_grad():
        outputs = model(input)
    _, preds = torch.max(outputs.logits, -1)
    preds = preds[0].cpu().numpy()
    return preds

In [17]:
sentence = """A 48 year - old female presented with vaginal bleeding and abnormal Pap smears .
Upon diagnosis of invasive non - keratinizing SCC of the cervix ,
she underwent a radical hysterectomy with salpingo - oophorectomy
which demonstrated positive spread to the pelvic lymph nodes and the parametrium .
Pathological examination revealed that the tumour also extensively involved the lower uterine segment .
"""

preds = inference(sentence, model)
results = merge_entity(sentence, preds, model)
results

[('O', 'A'),
 ('O', '48'),
 ('Age', 'year'),
 ('Sex', '-'),
 ('O', 'old'),
 ('O', 'female'),
 ('O', 'presented'),
 ('Sign_symptom', 'with'),
 ('O', 'vaginal'),
 ('Detailed_description', 'bleeding'),
 ('O', 'and'),
 ('O', 'abnormal'),
 ('O', 'Pap'),
 ('O', 'smears'),
 ('O', '.'),
 ('Detailed_description', 'Upon'),
 ('O', 'diagnosis'),
 ('O', 'of'),
 ('O', 'invasive'),
 ('O', 'non'),
 ('O', '-'),
 ('O', 'keratinizing'),
 ('O', 'SCC'),
 ('O', 'of'),
 ('O', 'the'),
 ('O', 'cervix'),
 ('O', ','),
 ('O', 'she'),
 ('O', 'underwent'),
 ('O', 'a'),
 ('Lab_value', 'radical'),
 ('O', 'hysterectomy'),
 ('O', 'with'),
 ('O', 'salpingo'),
 ('O', '-'),
 ('O', 'oophorectomy'),
 ('O', 'which'),
 ('O', 'demonstrated'),
 ('O', 'positive'),
 ('O', 'spread'),
 ('Diagnostic_procedure', 'to'),
 ('O', 'the'),
 ('O', 'pelvic'),
 ('O', 'lymph'),
 ('O', 'nodes'),
 ('O', 'and'),
 ('O', 'the'),
 ('O', 'parametrium'),
 ('O', '.'),
 ('O', 'Pathological'),
 ('O', 'examination'),
 ('O', 'revealed'),
 ('O', 'that')]